In [1]:
import sys
sys.path.append('../..')
sys.path.append('../data')
sys.path.append('../../helper_code')

from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
import copy

from helper_code.custom_kernel import *

In [3]:
input_dataset = ['c', 'c_lexi', 'c_lexi_nd', 'CE', 'CE_lexi', 'CE_lexi_nd', 'CSE', 'CSE_lexi', 'CSE_lexi_nd']
dataset_dict = {}

for data in input_dataset:
    dataset_dict[data] = pd.read_csv(f'../data/coronene_training_data/{data}.csv')

delta_total_energy = pd.read_csv(f'../data/coronene_training_data/delta_total_energy.csv')
delta_delta_total_energy = pd.read_csv(f'../data/coronene_training_data/delta_delta_total_energy.csv')

## CSE with Gaussian Kernel ##

In [4]:
X_train = dataset_dict['CSE_lexi'].to_numpy()
y_train = delta_delta_total_energy

params = {'alpha': 4.6e-11, 'gamma': 2.8e-08, 'kernel': 'rbf'}
model = KernelRidge(**params)

kfold = KFold(n_splits=2, shuffle=True, random_state=42)
neg_mse_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kfold)
rmse_scores = np.sqrt(-neg_mse_scores)
mean_rmse_score = rmse_scores.mean()

for fold, score in enumerate(rmse_scores):
    print(f"fold {fold}: rmse = {score}")

print(f"Average rmse: {mean_rmse_score}")

fold 0: rmse = 0.1488226861196492
fold 1: rmse = 0.1490829357396042
Average rmse: 0.1489528109296267


In [6]:
display(dataset_dict['CSE_lexi_nd'])

,coord0,coord1,coord2,coord3,coord4,coord5,coord6,coord7,coord8,coord9,...,coord14,coord15,coord16,coord17,coord18,coord19,coord20,coord21,coord22,coord23
0,2.2,-3.677288e-06,0.886896,0.030931,1.864525,0.019702,2.969533e-01,0.889326,4.846423,0.339437,...,0.009773,1.425104,0.188320,1.222189,3.097188e+00,2.915659e+00,0.487331,2.563553,0.107685,0.026267
1,1.2,-3.448928e-04,0.193056,0.271505,1.180707,0.108347,5.240912e-01,0.395139,0.107130,0.497755,...,0.003727,0.270157,0.067595,0.303747,2.046110e-08,1.293816e+00,1.834507,0.101007,4.045417,0.016284
2,2.0,-2.774404e-04,2.799053,0.184836,1.785080,0.000607,3.208155e-02,0.098864,0.337784,0.626369,...,0.057853,2.318309,1.113983,0.000431,1.115304e+00,3.244921e-01,0.331819,1.386220,0.241169,0.769288
3,1.2,-2.095629e-13,2.003122,0.020422,2.388762,0.001236,7.808520e-09,1.581356,1.436274,0.273553,...,1.342402,0.046916,0.132920,1.459136,8.079706e-09,2.889529e-08,0.350950,0.204177,2.422035,0.258557
4,1.8,-8.625202e-05,1.316121,0.027750,0.008348,1.668350,1.307179e-01,3.558400,3.978830,0.163200,...,3.540344,0.065713,1.620760,0.278990,4.953462e-01,3.234549e-01,0.624602,0.023849,0.219413,0.005370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0.2,-6.729397e-04,0.097368,0.035985,0.214085,0.032452,2.927212e-01,0.098817,0.012621,0.001090,...,0.257690,0.000413,0.017023,0.000359,1.238689e-01,1.738714e-06,0.365206,0.325005,0.007516,0.107617
2396,2.0,-4.697928e-03,1.335999,0.001401,1.695804,2.588433,1.589297e+00,0.098865,0.761597,4.804520,...,0.100033,0.449693,0.126813,0.378975,1.115095e+00,3.275953e-01,0.047910,1.003294,0.008158,0.543534
2397,1.6,-6.730540e-04,0.689178,1.071490,1.519382,0.456226,2.925385e-01,0.098850,0.095692,0.120943,...,1.301361,0.618534,0.204931,0.640794,1.238546e-01,1.667698e-06,1.154817,5.779930,0.612162,0.013665
2398,0.8,-2.773914e-04,0.337727,0.020671,1.376142,0.056814,3.207955e-02,0.098907,1.079238,0.999166,...,0.117490,0.313514,0.084729,0.164279,1.238646e-01,3.249200e-01,0.206011,0.257901,3.813640,0.028562


In [9]:
X_train = dataset_dict['CSE'].to_numpy()
y_train = delta_delta_total_energy

params = {'alpha': 4.6e-11, 'gamma': 2.8e-08, 'kernel': 'rbf'}
model = KernelRidge(**params)

kfold = KFold(n_splits=2, shuffle=True, random_state=42)
neg_mse_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kfold)
rmse_scores = np.sqrt(-neg_mse_scores)
mean_rmse_score = rmse_scores.mean()

for fold, score in enumerate(rmse_scores):
    print(f"fold {fold}: rmse = {score}")

print(f"Average rmse: {mean_rmse_score}")

fold 0: rmse = 0.7333172896586568
fold 1: rmse = 0.70772046532868
Average rmse: 0.7205188774936684


## Pure Extended Gaussian ##

In [5]:
X_train = dataset_dict['c_lexi'].to_numpy()
y_train = delta_delta_total_energy.to_numpy()

params = {'gamma': 1e-8, 'epsilon': 4e-4, 'beta': 2e-07, 'alpha': 1e-10}
alpha = params['alpha']

similarity_matrix = vectorized_extended_gaussian_kernel(X_train, X_train, params)
print(similarity_matrix.shape)
# krr_model = KernelRidge(kernel='precomputed', alpha=alpha)
# kfold = KFold(n_splits=2, shuffle=True, random_state=42)

# with warnings.catch_warnings():
#     warnings.filterwarnings('ignore')
#     mse_scores = cross_val_score(krr_model, similarity_matrix, y_train, scoring='neg_mean_squared_error', cv=kfold)
#     rmse_scores = np.sqrt(-mse_scores)
#     avg_rmse = rmse_scores.mean()

# for fold, rmse in enumerate(rmse_scores):
#     print(f"Fold {fold+1}: RMSE = {rmse}")
# print(f"Average RMSE across all folds: {avg_rmse}")

: 

: 